In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import rc
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch import nn, optim
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error #as mae
from sklearn.metrics import r2_score
from statistics import mode
#from sklearn.preprocessing import MinMaxScaler
from pandas.plotting import register_matplotlib_converters
import copy
import os
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf # version tensorflow==2.4.0
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import random
random.seed(500)
import scipy.stats
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Conv3D
from keras.layers import ConvLSTM2D
from keras.layers import ConvLSTM3D
from keras.layers import BatchNormalization


Mounted at /content/drive


In [ ]:
#PreProcessing

In [ ]:
import os
os.chdir(r"/content/drive/My Drive")

In [ ]:
#Test dataset from LSTNet
from google.colab import files
uploaded = files.upload()

Saving electricity datasets.csv to electricity datasets.csv


In [ ]:
#datasets = pd.read_csv('electricity datasets.csv',encoding='ISO-8859-1')
#datasets
#datasets = pd.read_csv('ETTh1.csv')
#datasets = pd.read_csv('ETTh2.csv')
#datasets = pd.read_csv('ETTm1.csv')
#datasets = pd.read_csv('ETTm2.csv')
#datasets = pd.read_csv('energydata_complete.csv')
datasets = pd.read_csv('exchange_rate.csv')
datasets

,date,0,1,2,3,4,5,6,OT
0,01/01/1990 00:00,0.785500,1.611000,0.861698,0.634196,0.211242,0.006838,0.525486,0.593000
1,02/01/1990 00:00,0.781800,1.610000,0.861104,0.633513,0.211242,0.006863,0.523972,0.594000
2,03/01/1990 00:00,0.786700,1.629300,0.861030,0.648508,0.211242,0.006975,0.526316,0.597300
3,04/01/1990 00:00,0.786000,1.637000,0.862069,0.650618,0.211242,0.006953,0.523834,0.597000
4,05/01/1990 00:00,0.784900,1.653000,0.861995,0.656254,0.211242,0.006940,0.527426,0.598500
...,...,...,...,...,...,...,...,...,...
7583,06/10/2010 00:00,0.718494,1.222195,0.737485,0.969974,0.143697,0.008500,0.688565,0.690846
7584,07/10/2010 00:00,0.721839,1.223459,0.741155,0.977297,0.143763,0.008595,0.690288,0.695701
7585,08/10/2010 00:00,0.723197,1.234111,0.745184,0.984446,0.143997,0.008562,0.691419,0.695943
7586,09/10/2010 00:00,0.720825,1.233905,0.744131,0.980344,0.143993,0.008555,0.690942,0.692689


In [ ]:
#datasets['date'] = pd.to_datetime(datasets['date']).dt.date
datasets['date'] = pd.to_datetime(datasets['date'],format='mixed').dt.date

datasets

,date,0,1,2,3,4,5,6,OT
0,1990-01-01,0.785500,1.611000,0.861698,0.634196,0.211242,0.006838,0.525486,0.593000
1,1990-02-01,0.781800,1.610000,0.861104,0.633513,0.211242,0.006863,0.523972,0.594000
2,1990-03-01,0.786700,1.629300,0.861030,0.648508,0.211242,0.006975,0.526316,0.597300
3,1990-04-01,0.786000,1.637000,0.862069,0.650618,0.211242,0.006953,0.523834,0.597000
4,1990-05-01,0.784900,1.653000,0.861995,0.656254,0.211242,0.006940,0.527426,0.598500
...,...,...,...,...,...,...,...,...,...
7583,2010-06-10,0.718494,1.222195,0.737485,0.969974,0.143697,0.008500,0.688565,0.690846
7584,2010-07-10,0.721839,1.223459,0.741155,0.977297,0.143763,0.008595,0.690288,0.695701
7585,2010-08-10,0.723197,1.234111,0.745184,0.984446,0.143997,0.008562,0.691419,0.695943
7586,2010-09-10,0.720825,1.233905,0.744131,0.980344,0.143993,0.008555,0.690942,0.692689


In [ ]:
datasets['date']=pd.to_datetime(datasets['date'])

In [ ]:
datasets.index = datasets.pop('date')

In [ ]:
datasets

,0,1,2,3,4,5,6,OT
date,,,,,,,,
1990-01-01,0.785500,1.611000,0.861698,0.634196,0.211242,0.006838,0.525486,0.593000
1990-02-01,0.781800,1.610000,0.861104,0.633513,0.211242,0.006863,0.523972,0.594000
1990-03-01,0.786700,1.629300,0.861030,0.648508,0.211242,0.006975,0.526316,0.597300
1990-04-01,0.786000,1.637000,0.862069,0.650618,0.211242,0.006953,0.523834,0.597000
1990-05-01,0.784900,1.653000,0.861995,0.656254,0.211242,0.006940,0.527426,0.598500
...,...,...,...,...,...,...,...,...
2010-06-10,0.718494,1.222195,0.737485,0.969974,0.143697,0.008500,0.688565,0.690846
2010-07-10,0.721839,1.223459,0.741155,0.977297,0.143763,0.008595,0.690288,0.695701
2010-08-10,0.723197,1.234111,0.745184,0.984446,0.143997,0.008562,0.691419,0.695943


In [ ]:
#Energy dataset
#target = datasets['RH_out']
#target =pd.DataFrame(target,columns=['RH_out'])
#target
#ETT dataset
#target = datasets['OT']
#target =pd.DataFrame(target,columns=['OT'])
#target
#ExchangeRate dataset
target = datasets['OT']
target =pd.DataFrame(target,columns=['OT'])
target

,OT
date,
1990-01-01,0.593000
1990-02-01,0.594000
1990-03-01,0.597300
1990-04-01,0.597000
1990-05-01,0.598500
...,...
2010-06-10,0.690846
2010-07-10,0.695701
2010-08-10,0.695943


In [ ]:
#For ETTm Dataset
#test_data_size = 13936
#For ETTh Dataset
#test_data_size = 3484
#For Energy Dataset
#test_data_size = 3947
#For Exchange Dataset
test_data_size = 3484
train_data_target = target[:-test_data_size]
test_data_target = target[-test_data_size:]


In [ ]:
print("Len Train Data Target",train_data_target.shape)
print("Len Test Data Target",test_data_target.shape)

Len Train Data Target (4104, 1)
Len Test Data Target (3484, 1)


In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_target = scaler.fit_transform(train_data_target)
test_data_target = scaler.fit_transform(test_data_target)

In [ ]:
def create_sequences(data, seq_length):
    xs = []
    ys = []

    for i in range(len(data)-seq_length-1):
        x = data[i:(i+seq_length)]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)

    return np.array(xs), np.array(ys)

In [ ]:
seq_length = 24
#Target
X_train_target, y_train_target = create_sequences(train_data_target, seq_length)
X_test_target, y_test_target = create_sequences(test_data_target, seq_length)
X_train_target = torch.from_numpy(X_train_target).float()
y_train_target = torch.from_numpy(y_train_target).float()
X_test_target = torch.from_numpy(X_test_target).float()
y_test_target = torch.from_numpy(y_test_target).float()

In [ ]:
print("X train target",X_train_target.shape)
print("y train target",y_train_target.shape)
print("-------------------------------------")

X train target torch.Size([4079, 24, 1])
y train target torch.Size([4079, 1])
-------------------------------------


In [ ]:
print("X test target",X_test_target.shape)
print("y test target",y_test_target.shape)
print("-------------------------------------")

X test target torch.Size([3459, 24, 1])
y test target torch.Size([3459, 1])
-------------------------------------


In [ ]:
class DeepConvLSTM:
  def __init__(self, epochs, batch_size, learning_rate,beta_1,beta_2):
    self.epochs = epochs
    self.batch_size = batch_size
    self.learning_rate = learning_rate
    self.beta_1 = beta_1
    self.beta_2 = beta_2

  def Tensor(self):
    self.X_target = np.array(X_train_target)
    self.Y_target = np.array(y_train_target)
    self.test_X_target = np.array(X_test_target)
    self.test_Y_target = np.array(y_test_target)
    #self.test_Y_ConvLSTM = np.array(y_test_target)
    #test_X_ConvLSTM = np.array(X_test_target)
    #test_Y_ConvLSTM = np.array(y_test_target)

  def ConvLSTM(self):
    #Construct Model
    model_ConvLSTM = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=32, kernel_size=3,
                               strides=1, padding="causal",
                               activation="relu",
                              input_shape=(X_train_target.shape[1],X_train_target.shape[2])),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8,return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(4,return_sequences=False)),
        tf.keras.layers.Dense(1),
        tf.keras.layers.Lambda(lambda x: x * 100)]) #original value 200
    #optimizer = tf.keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9) #3e-4,1e-5
    #model_TargetConvLSTM.compile(loss=tf.keras.losses.Huber(),optimizer=optimizer,metrics=['mse'])
    optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate, beta_1=self.beta_1, beta_2=self.beta_2,amsgrad=True)
    model_ConvLSTM.compile(loss='mean_squared_error', optimizer = optimizer)
    #Train The Model
    ConvLSTM = model_ConvLSTM.fit(self.X_target,self.Y_target,shuffle = False,epochs=self.epochs,batch_size=self.batch_size,validation_split=0.2,verbose=1)
    #Predict
    self.y_pred_ConvLSTM = model_ConvLSTM.predict(self.test_X_target)[:24] #[:720] #[:336] #[:192] #[:96]
    self.final_predict = self.y_pred_ConvLSTM.flatten()[:24]
    self.test_Y_target = self.test_Y_target.flatten()[:24]

    #def Ploting_train(self):
    #  pass
  def Final_predict(self):
    #self.y_pred_BiLSTM = self.y_pred_BiLSTM.flatten()[:720] #[:336] #[:192] #[:96]      # Predict AQI using ConvLSTM
    self.y_pred_ConvLSTM = self.y_pred_ConvLSTM.flatten()[:24] #[:720] #[:336] #[:192] #[:96]          # predict AQI using Bi-LSTM
    #self.y_pred_StakedLSTM = self.y_pred_StakedLSTM.flatten()[:720] #[:336] #[:192] #[:96]            # Predict ExFactor
    #y_pred_StackedGRU = y_pred_StackedGRU.flatten()  # Predict Ratio
    #y_pred_StakedLSTM = y_pred_StakedLSTM.flatten()
    #self.test_Y_target = self.test_Y_target.flatten()[:720] #[:336] #[:192] #[:96]
    #yHat = (y_pred_ConvLSTM + y_pred_BiLSTM + y_pred_BiGRU + y_pred_StackedGRU) / 4
    #self.yHat = (self.y_pred_BiLSTM + self.y_pred_ConvLSTM + self.y_pred_StakedLSTM) / 3
    #yHat2 = (((y_pred_ConvLSTM + y_pred_BiLSTM + y_pred_BiGRU) / 3 ) + y_pred_StackedGRU)/2
    #yHat3 = (((y_pred_ConvLSTM + y_pred_BiLSTM + y_pred_StackedGRU) / 3 ) + y_pred_BiGRU)/2

  def Dataframe(self):
    self.result = pd.DataFrame(data={'Actual AQI':self.test_Y_target,'Deep ConvLSTM':self.final_predict})
    return self.result

  def smape(self,a, f):
    return 1/len(self.a) * np.sum(2 * np.abs(f-a) / (np.abs(a) + np.abs(f))*100)

  def matrik_error(self):
    self.mse_ensemble = mean_squared_error(self.result['Actual AQI'], self.result['Deep ConvLSTM'])
    self.mae_ensemble = mean_absolute_error(self.result['Actual AQI'], self.result['Deep ConvLSTM'])
    #smape_ensemble = smape(first_strategy_result['Actual AQI'], first_strategy_result['Ensemble Methods'])
    self.r2_ensemble = r2_score(self.result['Actual AQI'], self.result['Deep ConvLSTM'],multioutput='variance_weighted')

  def evaluation(self):
    print("MSE First Strategy",self.mse_ensemble)
    print("MAE First Strategy",self.mae_ensemble)
    #print("Smape First Strategy",smape_ensemble)
    print("R2 First Strategy",self.r2_ensemble)



In [ ]:
Model = DeepConvLSTM(12,8,0.0001,0.9,0.99)
Model.Tensor()
#Model.BiLSTM()
Model.ConvLSTM()
#Model.StackedLSTM()
#Model.Final_predict()
Model.Dataframe()
Model.matrik_error()
Model.evaluation()

Epoch 1/12
408/408 [==============================] - 37s 44ms/step - loss: 0.0577 - val_loss: 10.1965
Epoch 2/12
408/408 [==============================] - 12s 30ms/step - loss: 0.0227 - val_loss: 1.4537
Epoch 3/12
408/408 [==============================] - 12s 29ms/step - loss: 0.0260 - val_loss: 16.2775
Epoch 4/12
408/408 [==============================] - 12s 28ms/step - loss: 0.0771 - val_loss: 5.4920
Epoch 5/12
408/408 [==============================] - 12s 30ms/step - loss: 0.1259 - val_loss: 1.6463
Epoch 6/12
408/408 [==============================] - 13s 31ms/step - loss: 0.0799 - val_loss: 1.1123
Epoch 7/12
408/408 [==============================] - 12s 31ms/step - loss: 0.0307 - val_loss: 1.2877
Epoch 8/12
408/408 [==============================] - 12s 30ms/step - loss: 0.0177 - val_loss: 1.7033
Epoch 9/12
408/408 [==============================] - 13s 31ms/step - loss: 0.0138 - val_loss: 2.1804
Epoch 10/12
408/408 [==============================] - 13s 31ms/step - loss: 0.0

In [ ]:
n = 17
print(type(n))
pi = 3.14159265
print(type(pi))
messages = 'ini adalah string'
print(type(messages))
tes = 123

<class 'int'>
<class 'float'>
<class 'str'>
